In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# Load the CSV file
csv_file = '/Users/sidchaudhary/Documents/GitHub/Hydro-Seesaw/Results/lat_lon_flag_counts_mid_end_season_with_early_baseline.csv'
df = pd.read_csv(csv_file)

# Convert lat/lon to geometry points
df['geometry'] = df.apply(lambda row: Point(row['long'], row['lat']), axis=1)
gdf_points = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')  # Assuming lat/lon are in EPSG:4326




In [ ]:
# Load the shapefile
shapefile = '/Users/sidchaudhary/Documents/GitHub/Hydro-Seesaw/data/IPCC-WGI-reference-regions-v4_shapefile/IPCC-WGI-reference-regions-v4.shp'
gdf_polygons = gpd.read_file(shapefile)

# Reproject points to match the shapefile CRS if necessary
if gdf_points.crs != gdf_polygons.crs:
    gdf_points = gdf_points.to_crs(gdf_polygons.crs)

# Spatial join to find which points fall inside which polygon
joined_gdf = gpd.sjoin(gdf_points, gdf_polygons, how='inner', op='within')

# Group by polygon, scenario, time_period, and season, and calculate mean values for the flag columns
flag_columns = ['p1_flag', 'p5_flag', 'p10_flag', 'p90_flag', 'p95_flag', 'p99_flag']
grouped = joined_gdf.groupby(['index_right', 'scenario', 'time_period', 'season'])[flag_columns].mean().reset_index()

# Round the values to 2 decimal places
grouped[flag_columns] = grouped[flag_columns].round(2)

# Merge back with polygon information
grouped_with_polygons = pd.merge(grouped, gdf_polygons, left_on='index_right', right_index=True)

# Save output as separate CSVs for each combination of time_period, scenario, and season
for time_period in grouped['time_period'].unique():
    for scenario in grouped['scenario'].unique():
        for season in grouped['season'].unique():
            subset = grouped_with_polygons[(grouped_with_polygons['time_period'] == time_period) & 
                                           (grouped_with_polygons['scenario'] == scenario) & 
                                           (grouped_with_polygons['season'] == season)]
            # Save each combination as a separate CSV
            #subset.to_csv(f'/Users/sidchaudhary/Documents/GitHub/Hydro-Seesaw/Results/{time_period}_{scenario}_{season}.csv', index=False)

# Save aggregated results in a single file
grouped_with_polygons.to_csv('/Users/sidchaudhary/Documents/GitHub/Hydro-Seesaw/Results/aggregated_results_with_early_baseline.csv', index=False)

/Users/sidchaudhary/opt/anaconda3/envs/streamflow/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
